### Deploy and Interact with LUCE contract
The aim of this notebook is to compile, deploy and interact with all functions of the LUCE smart contract (v3).

### Compile contract from file

In [1]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

Using solc version v0.4.25


In [2]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [3]:
# Compile & Store Compiled source code
compiled_sol = compile_source(contract_source_code)

In [4]:
#compiled_sol

In [5]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [9]:
#contract_interface

In [10]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [11]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [12]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [13]:
# Create contract blueprint
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [14]:
# Submit the transaction that deploys the contract
tx_hash = Luce.constructor().transact()

### Obtain Transcation Receipt

In [15]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [17]:
# Obtain address of freshly deployed contract
tx_receipt.contractAddress

'0x392e7cc6C9Ed71958Bee664FE3C0844B72b69b8A'

### Interact with contract

In [18]:
# Create python instance of deployed contract
luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

**publishData**  
```solidity
publishData(string memory _newdescription, string memory _link, uint _licence)
```

In [21]:
# Publish Data
tx_hash = luce1.functions.publishData("example description", "my link", 6).transact()

**setLicence**
```solidity
setLicence(uint newLicence)
```

In [22]:
# Set Licence
_ = luce1.functions.setLicence(28).transact()

From here on I am using `_` to suppress the output of the transaction hash. I do this since transactions are mined instantaneously in the local ganache test setting. On the testnet and mainnet we would store the transaction hash, then wait until a transaction receipt is available and only then we can be sure that the transaction went through.

**getLicence**
```solidity
getLicence() public view returns(uint)
```

In [23]:
# Get Licence
luce1.functions.getLicence().call()

28

**updateData**
```solidity
updateData(string memory updateDescr, string memory _newlink)
```

In [24]:
# Update Data
_ = luce1.functions.updateData("New Description", "New Link").transact()

**getLink**
```solidity
getLink(uint token) public view returns(string memory)
```

In [26]:
luce1.functions.getLink(1).call()

'New Link'

All LUCE functions up to this point are working.

---

We can also query the dataProvider and dataDescription since public variables in smart contracts automatically have a getter method. We simply call them by their name to read the current variable state:

**dataProvider**

In [28]:
luce1.functions.dataProvider().call()

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

**dataDescription**

In [29]:
luce1.functions.dataDescription().call()

'New Description'

---

The following two functions are not working yet and need debugging:

**addDataRequester**
```solidity
addDataRequester(uint purposeCode, uint licenceType) public returns(uint)
```

In [30]:
luce1.functions.addDataRequester(1,2).transact()

ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'0x35c443d9a91affeefdc638fd16db80517679658afb715abc057ebc1b35fc70d5': {'error': 'revert', 'program_counter': 3312, 'return': '0x'}, 'stack': 'o: VM Exception while processing transaction: revert\n    at Function.o.fromResults (/home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:10:81931)\n    at /home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:115772\n    at /home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:96183\n    at p (/home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:95840)\n    at /home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:95893\n    at t.default (/home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganach

This method throws an error at the moment:  
`'message': 'VM Exception while processing transaction: revert`

**renewToken**
```solidity
renewToken(uint compliance) public returns(uint token)
```

In [31]:
luce1.functions.renewToken(1).transact()
# Error: Need to agree on licence first

ValueError: {'message': 'VM Exception while processing transaction: revert need to agree on licence first', 'code': -32000, 'data': {'0x9b8d3507615ff3c250b052ac86f8f5b6c4888aef9b03ee8fb33b5b39e680c7a2': {'error': 'revert', 'program_counter': 2060, 'return': '0x08c379a00000000000000000000000000000000000000000000000000000000000000020000000000000000000000000000000000000000000000000000000000000001e6e65656420746f206167726565206f6e206c6963656e63652066697273740000', 'reason': 'need to agree on licence first'}, 'stack': 'o: VM Exception while processing transaction: revert need to agree on licence first\n    at Function.o.fromResults (/home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:10:81931)\n    at /home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:115772\n    at /home/vagrant/.nvm/versions/node/v12.4.0/lib/node_modules/ganache-cli/build/ganache-core.node.cli.js:47:96183\n    at p (/home/

`'message': 'VM Exception while processing transaction: revert need to agree on licence first'`